In [1]:
import os
import cv2
import json
import numpy as np
import pandas as pd
from PIL import Image
import tensorflow as tf
from itertools import chain
from tensorflow import keras
from ultralytics import YOLO
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from keras.utils import to_categorical
%matplotlib inline

**Showing Pictures**

In [2]:
image_path = r"D:\uni\Human Detection\Test\8.jpg"
image = cv2.imread(image_path)
if image is not None:
    cv2.imshow("Image", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print(f"Unable to read the image at {image_path}.")

**Adding Pre-trained Model**

In [3]:
model = YOLO(r"D:\uni\Human Detection\Models\yolov8m.pt")
pose_model = YOLO(r"D:\uni\Human Detection\Models\yolov8n-pose.pt")

**Model Result**

In [13]:
if image is not None:
    results = model.predict(image, verbose=False)
    results = results[0]

    person_count = 0

    for box in results.boxes:
        class_id = results.names[box.cls[0].item()]
        cords = box.xyxy[0].tolist()
        cords = [round(x) for x in cords]
        conf = round(box.conf[0].item(), 2)

        if class_id == "person" and conf >= 0.30:
            person_count += 1
            x1, y1, x2, y2 = cords
            cv2.rectangle(image, (x1, y1), (x2, y2), (69, 145, 31), 3)
            
            bg_width, bg_height = 50, 25
            text = f"{conf}%"
            cv2.rectangle(image, (x1, y1 - bg_height), (x1 + bg_width, y1), (69, 145, 31), -1)
            cv2.putText(image, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
            
    total_bg_width, total_bg_height = 370, 60
    text_total = f"{person_count} Person(s) Detected."
    cv2.rectangle(image, (0, 40 - total_bg_height), (20 + total_bg_width, 40), (0, 0, 0), -1)
    cv2.putText(image, text_total, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (69, 145, 31), 3)
    
    cv2.imshow("Result", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print(f"Unable to read the image at {image_path}.")

In [5]:
from keras.models import load_model
loaded_model = load_model(r"D:\uni\Human Detection\Models\human_pose_detector.h5")

In [6]:
folder_path = r"D:\uni\Human Detection\Test"
images = []
for filename in os.listdir(folder_path):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        image=cv2.resize(img, (28,28))
        images.append(image)

x_test = np.array(images)

In [7]:
y_predict = loaded_model.predict(x_test)

1/1 [==============================] - 0s 264ms/step


In [8]:
print(y_predict)
y_predict.argmax()

[[    0.99494  0.00090808   0.0024102  3.6206e-05   0.0017084]
 [  0.0040358  0.00021988  1.2114e-05     0.99497  0.00076269]
 [    0.10421      0.3574     0.43843   0.0021873    0.097768]
 [    0.47534     0.30651    0.019977     0.19802  0.00014626]
 [  0.0072704   0.0080036     0.95554  1.3314e-07    0.029184]
 [   0.017749     0.58178    0.012788      0.3872  0.00047562]
 [  0.0010982    0.053045     0.93989   0.0055352  0.00043535]
 [    0.98433   0.0025365  0.00054475   0.0015818    0.011008]
 [    0.93311  0.00033495     0.06645  1.0048e-08  0.00010531]
 [  0.0023524  0.00090944   0.0019317  0.00013382     0.99467]]


8

In [9]:
arr = y_predict.argmax(axis=1)
arr_as_strings = [str(num) for num in arr]
print(arr_as_strings)

['0', '3', '2', '0', '2', '1', '2', '0', '0', '4']


In [10]:
arr = y_predict.argmax(axis=1)
label_map = {0: 'walking', 1: 'sitting', 2: 'standing', 3: 'leaning', 4: 'drinking'}

arr_as_strings = [label_map[num] for num in arr]
print(arr_as_strings)

['walking', 'leaning', 'standing', 'walking', 'standing', 'sitting', 'standing', 'walking', 'walking', 'drinking']


In [11]:
BODY_PARTS = {"Nose": 0, "Neck": 1, "Right Shoulder": 2, "Right Elbow": 3, "Right Wrist": 4,
              "Left Shoulder": 5, "Left Elbow": 6, "Left Wrist": 7, "Right Hip": 8, "Right Knee": 9,
              "Right Ankle": 10, "Left Hip": 11, "Left Knee": 12, "Left Ankle": 13, "Right Eye": 14,
              "Left Eye": 15, "Right Ear": 16, "Left Ear": 17, "Background": 18}

POSE_PAIRS = [["Neck", "Right Shoulder"], ["Neck", "Left Shoulder"], ["Right Shoulder", "Right Elbow"],
              ["Right Elbow", "Right Wrist"], ["Left Shoulder", "Left Elbow"], ["Left Elbow", "Left Wrist"],
              ["Neck", "Right Hip"], ["Right Hip", "Right Knee"], ["Right Knee", "Right Ankle"], ["Neck", "Left Hip"],
              ["Left Hip", "Left Knee"], ["Left Knee", "Left Ankle"], ["Neck", "Nose"], ["Nose", "Right Eye"],
              ["Right Eye", "Right Ear"], ["Nose", "Left Eye"], ["Left Eye", "Left Ear"]]

inWidth = 368
inHeight = 368

net = cv2.dnn.readNetFromTensorflow(r"D:\uni\Human Detection\Models\graph_opt.pb")

uploaded_pictures = {"image1.jpg": r"D:\uni\Human Detection\Test\8.jpg"}

for i, file_name in enumerate(uploaded_pictures.values()):
    frame = cv2.imread(file_name)
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]

    net.setInput(cv2.dnn.blobFromImage(frame, 1.0, (inWidth, inHeight), (127.5, 127.5, 127.5), swapRB=True, crop=False))
    out = net.forward()
    out = out[:, :19, :, :]

    assert len(BODY_PARTS) == out.shape[1]

    points = []
    for i in range(len(BODY_PARTS)):
        heatMap = out[0, i, :, :]
        _, conf, _, point = cv2.minMaxLoc(heatMap)
        x = int((frameWidth * point[0]) / out.shape[3])
        y = int((frameHeight * point[1]) / out.shape[2])
        points.append((x, y) if conf > 0.2 else None)

    for pair in POSE_PAIRS:
        partFrom = pair[0]
        partTo = pair[1]
        assert partFrom in BODY_PARTS
        assert partTo in BODY_PARTS

        idFrom = BODY_PARTS[partFrom]
        idTo = BODY_PARTS[partTo]

        if points[idFrom] and points[idTo]:
            cv2.line(frame, points[idFrom], points[idTo], (255, 0, 0), 3)
            cv2.ellipse(frame, points[idFrom], (3, 3), 0, 0, 360, (0, 0, 255), cv2.FILLED)
            cv2.ellipse(frame, points[idTo], (3, 3), 0, 0, 360, (0, 0, 255), cv2.FILLED)

    neck_point = points[BODY_PARTS["Neck"]]
    left_hip_point = points[BODY_PARTS["Left Hip"]]
    right_hip_point = points[BODY_PARTS["Right Hip"]]

    if neck_point and left_hip_point and right_hip_point:
        neck_y = neck_point[1]
        left_hip_y = left_hip_point[1]
        right_hip_y = right_hip_point[1]

        if left_hip_y < neck_y < right_hip_y:
            status = "Standing"
        else:
            status = "Walking"
    else:
        status = "leaning or sitting"

    cv2.putText(frame, status, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imshow("Pose", frame)
    cv2.waitKey(0)
cv2.destroyAllWindows() 


In [12]:
img_path = r"D:\uni\Human Detection\Test\8.jpg"
pose_result = pose_model(source=img_path, conf=0.5, show=True, save=True)
cv2.waitKey(0)
cv2.destroyAllWindows()


image 1/1 D:\uni\Human Detection\Test\8.jpg: 640x480 1 person, 255.0ms
Speed: 9.1ms preprocess, 255.0ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 480)
Results saved to runs\pose\predict5
